## Linear Regression Model

In [1]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt

# Generate sample data
np.random.seed(42)
n = 100  # number of data points
p = 3    # number of features (excluding intercept)

# True underlying relationship: y = 2 + 3*x1 + 1.5*x2 - 2*x3 + noise
X = np.random.randn(n, p)
true_beta = np.array([3, 1.5, -2])
true_intercept = 2
y = true_intercept + X @ true_beta + 0.5 * np.random.randn(n)

print("Data shape:", X.shape)
print("First 5 rows of X:\n", X[:5])
print("First 5 values of y:", y[:5])

Data shape: (100, 3)
First 5 rows of X:
 [[ 0.49671415 -0.1382643   0.64768854]
 [ 1.52302986 -0.23415337 -0.23413696]
 [ 1.57921282  0.76743473 -0.46947439]
 [ 0.54256004 -0.46341769 -0.46572975]
 [ 0.24196227 -1.91328024 -1.72491783]]
First 5 values of y: [1.57287143 6.4060429  9.20138611 4.16919823 3.29535132]


In [2]:
def linear_regression_ols(X, y):
    n, p = X.shape ## n is the number of data entries, and p is the number of features

    # Decision variables: coefficients (including intercept as first element)
    beta = cp.Variable(p) ## the index should be at [p]
    intercept = cp.Variable()

    # Define the objective: minimize sum of squared errors
    predictions = X @ beta + intercept   ## the intercept is added to each prediction equally, predictions is a n*1 vector
    objective = cp.Minimize(cp.sum_squares(y - predictions))

    # Form and solve the problem
    problem = cp.Problem(objective)
    problem.solve()

    print(f"Status: {problem.status}")
    print(f"Optimal value: {problem.value:.4f}")

    return intercept.value, beta.value

# Solve the OLS problem
intercept_ols, beta_ols = linear_regression_ols(X, y)

print("\n--- OLS Results ---")
print(f"True intercept: {true_intercept}, Estimated: {intercept_ols:.4f}")
print(f"True coefficients: {true_beta}")
print(f"Estimated coefficients: {np.round(beta_ols, 4)}")

Status: optimal
Optimal value: 18.9191

--- OLS Results ---
True intercept: 2, Estimated: 2.0564
True coefficients: [ 3.   1.5 -2. ]
Estimated coefficients: [ 2.9612  1.475  -2.0538]


## Test on Housing prediction task

In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the California housing dataset
housing = fetch_california_housing()
X, y = housing.data, housing.target
feature_names = housing.feature_names

print("Dataset shape:", X.shape) ## we have 20640 data entries, 8 features
print("Number of features:", X.shape[1])
print("Feature names:", feature_names)
print("Target: Median House Value (in $100,000s)")

Dataset shape: (20640, 8)
Number of features: 8
Feature names: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Target: Median House Value (in $100,000s)


In [4]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Standardize features (important for regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
print("\n--- Training Linear Regression Model ---")
intercept, beta = linear_regression_ols(X_train_scaled, y_train)

# Display learned coefficients
print("\n--- Learned Coefficients ---")
print(f"Intercept: ${intercept:,.2f}")
for i, (name, coef) in enumerate(zip(feature_names, beta)):
    print(f"{name}: {coef:,.2f}")

Training set: 16512 samples
Test set: 4128 samples

--- Training Linear Regression Model ---
Status: optimal
Optimal value: 8552.1118

--- Learned Coefficients ---
Intercept: $2.07
MedInc: 0.85
HouseAge: 0.12
AveRooms: -0.29
AveBedrms: 0.34
Population: -0.00
AveOccup: -0.04
Latitude: -0.90
Longitude: -0.87


In [5]:
# Make predictions on test set
def predict(X, beta, intercept):
    return X @ beta + intercept

y_pred = predict(X_test_scaled, beta, intercept)

# Evaluate model performance
def evaluate_predictions(y_true, y_pred):
    mse = np.mean((y_true - y_pred) ** 2)
    mae = np.mean(np.abs(y_true - y_pred))
    r2 = 1 - np.sum((y_true - y_pred) ** 2) / np.sum((y_true - np.mean(y_true)) ** 2)

    print(f"Mean Squared Error: ${mse:,.2f}")
    print(f"Mean Absolute Error: ${mae:,.2f}")
    print(f"R² Score: {r2:.4f}")

    return mse, mae, r2

print("\n--- Model Performance on Test Set ---")
mse, mae, r2 = evaluate_predictions(y_test, y_pred)

# Display some actual vs predicted values ## we can display the first 10 samples
print("\n--- Sample Predictions ---")
print("Actual vs Predicted Prices:")
for i in range(10):
    print(f"${y_test[i]:,.2f} -> ${y_pred[i]:,.2f} (Error: ${y_test[i]-y_pred[i]:,.2f})")


--- Model Performance on Test Set ---
Mean Squared Error: $0.56
Mean Absolute Error: $0.53
R² Score: 0.5758

--- Sample Predictions ---
Actual vs Predicted Prices:
$0.48 -> $0.72 (Error: $-0.24)
$0.46 -> $1.76 (Error: $-1.31)
$5.00 -> $2.71 (Error: $2.29)
$2.19 -> $2.84 (Error: $-0.65)
$2.78 -> $2.60 (Error: $0.18)
$1.59 -> $2.01 (Error: $-0.42)
$1.98 -> $2.65 (Error: $-0.66)
$1.57 -> $2.17 (Error: $-0.59)
$3.40 -> $2.74 (Error: $0.66)
$4.47 -> $3.92 (Error: $0.55)
